In [22]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

Import movies data from Kaggle

https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset


Inspirational articles:

https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system 

In [96]:
genome_scores_df = pd.read_csv("data/genome_scores.csv").dropna()
genome_tags_df = pd.read_csv("data/genome_tags.csv").dropna()
link_df = pd.read_csv("data/link.csv").dropna()
movie_df = pd.read_csv("data/movie.csv").dropna()
rating_df = pd.read_csv("data/rating.csv").dropna()
tag_df = pd.read_csv("data/tag.csv").dropna()

In [318]:
tdf = tag_df
print(tdf.shape)
print(tdf.columns)
tdf['tag'].unique()

(465548, 4)
Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')


array(['Mark Waters', 'dark hero', 'noir thriller', ..., 'circle k',
       'This movie should have been called \\\\"How Cocaine Ruined Disney\\"\\""',
       'topless scene'], dtype=object)

We need a baseline to compare our predictions on. Lets find the most popular movie.

In [47]:
data = pd.merge(movie_df,rating_df, on= 'movieId')

In [48]:
data = pd.merge(movie_df,rating_df)

In [49]:
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41


In [50]:
total_ratings = pd.DataFrame(data.groupby('title')['rating'].count())
total_ratings = total_ratings.sort_values('rating', ascending = False)
total_ratings.head(10)

,rating
title,
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
Star Wars: Episode IV - A New Hope (1977),54502
Braveheart (1995),53769
Terminator 2: Judgment Day (1991),52244
"Matrix, The (1999)",51334


In [94]:
ex_data = data.groupby('title')['rating'].median()
ex_data.columns = ['title', 'medians']

total_ratings = pd.DataFrame(data.groupby('title')['rating'].count()).reset_index()
total_ratings.columns = ['title', 'counts']

ex_data = pd.merge(ex_data,total_ratings, on = 'title')


In [87]:
ex_data.head(10)

,title,rating,counts
0,#chicagoGirl: The Social Network Takes on a Di...,3.00,3
1,$ (Dollars) (1971),3.00,24
2,$5 a Day (2008),3.00,39
3,$9.99 (2008),3.00,55
4,$ellebrity (Sellebrity) (2012),2.00,2
5,'71 (2014),3.50,35
6,'Hellboy': The Seeds of Creation (2004),3.00,87
7,"'Human' Factor, The (Human Factor, The) (1975)",2.25,2
8,'Neath the Arizona Skies (1934),2.25,22
9,'R Xmas (2001),3.00,19


In [177]:
ex_data = ex_data.sort_values(['rating', 'counts'], ascending = False)
ex_data['rat_score'] = ex_data['rating']* ex_data['counts']
ex_data = ex_data.sort_values(['rat_score'], ascending = False)
ex_data.reset_index(drop = True).head(10)

,title,rating,counts,rat_score
0,"Shawshank Redemption, The (1994)",5.0,63366,316830.0
1,Pulp Fiction (1994),4.5,67310,302895.0
2,Forrest Gump (1994),4.0,66172,264688.0
3,"Silence of the Lambs, The (1991)",4.0,63299,253196.0
4,Star Wars: Episode IV - A New Hope (1977),4.5,54502,245259.0
5,Jurassic Park (1993),4.0,59715,238860.0
6,"Matrix, The (1999)",4.5,51334,231003.0
7,Schindler's List (1993),4.5,50054,225243.0
8,Braveheart (1995),4.0,53769,215076.0
9,"Usual Suspects, The (1995)",4.5,47006,211527.0


We can see 'Shawshank Redeption, The (1994)' is the most popular movie considering popularity and rating scores

We will investigate the weighted Rating equation 

WR = (v\*R)/(v+m) + (m\*C)/(v+m)

v is number of votes, m is minimum votes required to be considered, R is the rating of the movie, and C is the over all average 

In [196]:
m=500
v = data['title'].value_counts()
v = v[v >= m].reset_index()



R = data.groupby('title')['rating'].mean().reset_index()
C = data['rating'].mean()

print(ndata.size)


0


In [146]:
wr = (v*R)/(v+m) +(m*C)/(v+m)
wr.head()

title
Pulp Fiction (1994)                 0.025996
Forrest Gump (1994)                 0.026439
Shawshank Redemption, The (1994)    0.027601
Silence of the Lambs, The (1991)    0.027630
Jurassic Park (1993)                0.029275
Name: count, dtype: float64

In [176]:
temp_df = pd.merge(v,R)
temp_df['wr1'] = temp_df['count']*temp_df['rating']/(temp_df['count']+m)
temp_df['wr2'] = m*C/(temp_df['count']+m)
temp_df['wr'] = temp_df['wr1'] + temp_df['wr2']
wrdf = temp_df.sort_values(['wr'], ascending = False).reset_index(drop = True)
wrdf.head(10)

,title,count,rating,wr1,wr2,wr
0,"Shawshank Redemption, The (1994)",63366,4.446990,4.412175,0.027601,4.439776
1,"Godfather, The (1972)",41355,4.364732,4.312591,0.042116,4.354707
2,"Usual Suspects, The (1995)",47006,4.334372,4.288753,0.037106,4.325859
3,Schindler's List (1993),50054,4.310175,4.267546,0.034869,4.302415
4,"Godfather: Part II, The (1974)",27398,4.275641,4.199011,0.063186,4.262197
5,Rear Window (1954),17449,4.271334,4.152348,0.098210,4.250558
6,Casablanca (1942),24349,4.258327,4.172643,0.070939,4.243582
7,Seven Samurai (Shichinin no samurai) (1954),11611,4.274180,4.097721,0.145551,4.243272
8,One Flew Over the Cuckoo's Nest (1975),29932,4.248079,4.178283,0.057925,4.236207
9,Dr. Strangelove or: How I Learned to Stop Worr...,23220,4.247287,4.157757,0.074316,4.232073


Looks like 'Shawshank Redeption, The (1994)' is still the top score for Weighted Rating (WR). 

However the Godfather is ranked 2 and the previous number 2, Pulp Fiction is not found in the top 10.

The next stage we will look at the 'genre' of each movie from the movies data base to see the similarities

In [302]:
movies = movie_df[movie_df['title'].isin(v['title'])].reset_index(drop = True)

def split_bar(row):
    row1 = row.split('|')
    return ' '.join(row1)

movies['sgenres'] = movies['genres'].apply(split_bar)
movies.tail(30)

,movieId,title,genres,sgenres
4465,106696,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,Adventure Animation Comedy Fantasy Musical Rom...
4466,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama,Comedy Crime Drama
4467,106916,American Hustle (2013),Crime|Drama,Crime Drama
4468,106918,"Secret Life of Walter Mitty, The (2013)",Adventure|Comedy|Drama,Adventure Comedy Drama
4469,106920,Her (2013),Drama|Romance|Sci-Fi,Drama Romance Sci-Fi
4470,107406,Snowpiercer (2013),Action|Drama|Sci-Fi,Action Drama Sci-Fi
4471,108190,Divergent (2014),Adventure|Romance|Sci-Fi|IMAX,Adventure Romance Sci-Fi IMAX
4472,108932,The Lego Movie (2014),Action|Adventure|Animation|Children|Comedy|Fan...,Action Adventure Animation Children Comedy Fan...
4473,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,Comedy Drama
4474,109487,Interstellar (2014),Sci-Fi|IMAX,Sci-Fi IMAX


In [303]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['sgenres'])
tfidf_matrix.shape
#4.5k movies, 21 words

(4495, 21)

In [304]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(4495, 4495)

In [314]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = movies[movies['title'] == title].index

    sim_scores = list(enumerate(cosine_sim[idx][0]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[:11]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return movies['title'].iloc[movie_indices]

get_recommendations('X-Men: Days of Future Past (2014)')


[(163, 1.0), (205, 1.0), (247, 1.0), (344, 1.0), (792, 1.0), (805, 1.0), (937, 1.0), (1163, 1.0), (1351, 1.0), (1358, 1.0)]


163                                     Waterworld (1995)
205             Star Wars: Episode IV - A New Hope (1977)
247                                       Stargate (1994)
344                                 Demolition Man (1993)
792     Star Wars: Episode V - The Empire Strikes Back...
805     Star Wars: Episode VI - Return of the Jedi (1983)
937            Star Trek III: The Search for Spock (1984)
1163                                 Lost in Space (1998)
1351                                Rocketeer, The (1991)
1358                                          Tron (1982)
1467                            Six-String Samurai (1998)
Name: title, dtype: object

There, we have created a simple recommendation engine based on a cosine variation on their genres